<a href="https://colab.research.google.com/github/PaletteofDesign/sql/blob/main/Copy_of_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://course_report_production.s3.amazonaws.com/rich/rich_files/rich_files/2470/s300/cd-logo-blue-600x600.png" alt="Coding Dojo Logo" class="center" height="50">

# SQL Exercises

*Make a copy of this notebook to edit!*
```
File > Save a copy in Drive
```
    
</center>

[SQLAlchemy](https://www.sqlalchemy.org/) is a wrapper for Python that allows us to write SQL queries in Python. We will use this to practice our SQL basics in Python.

First, we need to connect to a database. We will just read a table into SQL for this step. We will be creating a [SQLite](https://sqlite.org/index.html) database and loading two tables ([table 1](https://drive.google.com/file/d/18MgOmIHHafoubgp2d_7GM6GWroxb0vPe/view?usp=sharing)  and [table 2](https://drive.google.com/file/d/1lqW6qfPXpDWsQ6yMYpZPX7brVif-v_Ra/view?usp=sharing)) into that database.

The data (from [this source](https://data.world/sikkim/school)) describes details of various schools.

In [1]:
# imports
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# mount data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# load table 1
school = pd.read_csv('/content/School Details.csv')
school.head(2)

,DistrictName,EstablishmentCode,EstablishmentName,BRCName,CRCName,SchoolLevel,ManagementName,LocalityHabitation,SubdivisionName,ConstituencyName,RevenueBlockName,CivicBodyName,WardName,PostOfficeName,PhoneNo,FaxNo,EmailID,Website,LocatedInRuralOrUrban,DiseCode,EstablishedDate,DistrictHQDistance (in Km),CRCDistance (in Km),BACDistance (in Km),IsRecognized,RecognizedBy,ContactPersonName,LowestClass,HighestClass,BoardName
0,East,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Duga Brc,Central Pendam Government Senior Secondary School,PS,State Govt.,BUDANG,Gangtok,Rhenock,Central Pendam,Central Pendam,Sajong,Upper Pendam,NaN,NaN,NaN,NaN,Urban Area,NaN,28126,58.0,8.0,16.0,Recognized,HRDD,NaN,Class 1,Class 5,NaN
1,East,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Duga Brc,Central Pendam Government Senior Secondary School,SSS,State Govt.,CENTRAL PENDAM,Gangtok,West Pandam,Central Pendam,Central Pendam,Karmithang,Upper Pendam,3592263812,NaN,NaN,NaN,Urban Area,800401,15342,50.0,0.0,6.0,Recognized,HRDD,NaN,Class 1,Class 12,C.B.S.E.


In [6]:
# load table 2
facility = pd.read_csv('/content/School Facility Details.csv')
facility.head(2)

,EstablishmentCode,EstablishmentName,IsLibraryAvailable,AvailableBooksCount,IsFireExtgAvailable,ExtinguisherCount,HOIRoomAvailable,IsStaffRoomAvailable,NoofStaffRoom,IsLockerForStaffAvailable,IsSportsStoreRoomAvailable
0,S1034,(I.E.C) INDIAN EVANGALISTIC CRUSADE MISSION PR...,Not Available,0,Available,1,Available,Not Available,0,Not Available,Not Available
1,S0516,10TH MILE GOVERNMENT PRIMARY SCHOOL,Not Available,0,Not Available,0,Available,Available,1,Not Available,Not Available


In [7]:
# create sqlite engine
engine = create_engine('sqlite:///school.db', echo=True)
# create connection to engine
conn = engine.connect()

In [8]:
# add the dataframe as a table in sqlite
sqlite_table = "school_details"
school.to_sql(sqlite_table, conn, if_exists='fail')

sqlite_table2 = "school_facility_details"
facility.to_sql(sqlite_table2, conn, if_exists='fail')

2021-10-07 00:51:13,921 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("school_details")
2021-10-07 00:51:13,930 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-07 00:51:13,934 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("school_details")
2021-10-07 00:51:13,937 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-07 00:51:13,946 INFO sqlalchemy.engine.Engine 
CREATE TABLE school_details (
	"index" BIGINT, 
	"DistrictName" TEXT, 
	"EstablishmentCode" TEXT, 
	"EstablishmentName" TEXT, 
	"BRCName" TEXT, 
	"CRCName" TEXT, 
	"SchoolLevel" TEXT, 
	"ManagementName" TEXT, 
	"LocalityHabitation" TEXT, 
	"SubdivisionName" TEXT, 
	"ConstituencyName" TEXT, 
	"RevenueBlockName" TEXT, 
	"CivicBodyName" TEXT, 
	"WardName" TEXT, 
	"PostOfficeName" TEXT, 
	"PhoneNo" TEXT, 
	"FaxNo" TEXT, 
	"EmailID" TEXT, 
	"Website" TEXT, 
	"LocatedInRuralOrUrban" TEXT, 
	"DiseCode" TEXT, 
	"EstablishedDate" TEXT, 
	"DistrictHQDistance (in Km)" FLOAT, 
	"CRCDistance (in Km)" FLOAT, 
	"BACDistance (in 

## Aggregates

1. What is the average BACDistance (in Km)? Rename the ouptut column to "avg_bac_distance."

In [9]:
# Write your SQL query in this string
sql = """
SELECT AVG ("BACDistance (in Km)") AS avg_bac_distance
FROM school_details
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-10-07 00:58:18,542 INFO sqlalchemy.engine.Engine 
SELECT AVG ("BACDistance (in Km)") AS avg_bac_distance
FROM school_details

2021-10-07 00:58:18,546 INFO sqlalchemy.engine.Engine [raw sql] ()


,avg_bac_distance
0,12.681822


## GROUP BY

2. What is the average BACDistance (in Km) by School Level? Sort the results from highest average to lowest average.

In [10]:
# Write your SQL query in this string
sql = """
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
ORDER BY avg_bac_distance DESC
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-10-07 00:58:52,026 INFO sqlalchemy.engine.Engine 
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
ORDER BY avg_bac_distance DESC

2021-10-07 00:58:52,028 INFO sqlalchemy.engine.Engine [raw sql] ()


,SchoolLevel,avg_bac_distance
0,JHS,13.692647
1,PS,12.830787
2,SS,12.630872
3,SSS,10.863095
4,LPS,5.368421


## Having

3. Repeat the previous query, but only display results where the average distance is at least 10 km.

In [11]:
# Write your SQL query in this string
sql = """
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
HAVING avg_bac_distance >= 10
ORDER BY avg_bac_distance DESC
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-10-07 00:59:16,945 INFO sqlalchemy.engine.Engine 
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
HAVING avg_bac_distance >= 10
ORDER BY avg_bac_distance DESC

2021-10-07 00:59:16,950 INFO sqlalchemy.engine.Engine [raw sql] ()


,SchoolLevel,avg_bac_distance
0,JHS,13.692647
1,PS,12.830787
2,SS,12.630872
3,SSS,10.863095


## Joins

4. Join the two tables together (school_details and school_details). Start by displaying all of the columns, but limit the resulting rows to just 2.

In [12]:
# Write your SQL query in this string
sql = """
SELECT *
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
LIMIT 2
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-10-07 00:59:56,245 INFO sqlalchemy.engine.Engine 
SELECT *
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
LIMIT 2

2021-10-07 00:59:56,250 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,DistrictName,EstablishmentCode,EstablishmentName,BRCName,CRCName,SchoolLevel,ManagementName,LocalityHabitation,SubdivisionName,ConstituencyName,RevenueBlockName,CivicBodyName,WardName,PostOfficeName,PhoneNo,FaxNo,EmailID,Website,LocatedInRuralOrUrban,DiseCode,EstablishedDate,DistrictHQDistance (in Km),CRCDistance (in Km),BACDistance (in Km),IsRecognized,RecognizedBy,ContactPersonName,LowestClass,HighestClass,BoardName,index,EstablishmentCode,EstablishmentName,IsLibraryAvailable,AvailableBooksCount,IsFireExtgAvailable,ExtinguisherCount,HOIRoomAvailable,IsStaffRoomAvailable,NoofStaffRoom,IsLockerForStaffAvailable,IsSportsStoreRoomAvailable
0,0,East,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Duga Brc,Central Pendam Government Senior Secondary School,PS,State Govt.,BUDANG,Gangtok,Rhenock,Central Pendam,Central Pendam,Sajong,Upper Pendam,None,None,None,None,Urban Area,None,28126,58.0,8.0,16.0,Recognized,HRDD,None,Class 1,Class 5,None,121,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Available,100,Available,2,Available,Available,1,Not Available,Not Available
1,1,East,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Duga Brc,Central Pendam Government Senior Secondary School,SSS,State Govt.,CENTRAL PENDAM,Gangtok,West Pandam,Central Pendam,Central Pendam,Karmithang,Upper Pendam,3592263812,None,None,None,Urban Area,800401,15342,50.0,0.0,6.0,Recognized,HRDD,None,Class 1,Class 12,C.B.S.E.,137,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Available,0,Available,5,Available,Available,2,Not Available,Available


5. Now, select just the schools where the SchoolLevel is "PS" that do not have libraries.

In [13]:
# Write your SQL query in this string
sql = """
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-10-07 01:00:31,904 INFO sqlalchemy.engine.Engine 
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'

2021-10-07 01:00:31,907 INFO sqlalchemy.engine.Engine [raw sql] ()


,EstablishmentName,SchoolLevel,IsLibraryAvailable
0,EASWARAMMA SAI GURUKOOL ACCADEMY,PS,Not Available
1,NEW GARDEN BOARDING SCHOOL (SAJONG),PS,Not Available
2,"SARASWATI SHISHU VIDHYALAYA, BUDANG",PS,Not Available
3,SAWNEY GOVERNMENT PRIMARY SCHOOL,PS,Not Available
4,UPPER PACHAK GOVERNMENT PRIMARY SCHOOL,PS,Not Available


6. Build off the previous query, but order the results alphabetically by name.

In [14]:
# Write your SQL query in this string
sql = """
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'
ORDER BY s.EstablishmentName
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-10-07 01:00:34,097 INFO sqlalchemy.engine.Engine 
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'
ORDER BY s.EstablishmentName

2021-10-07 01:00:34,102 INFO sqlalchemy.engine.Engine [raw sql] ()


,EstablishmentName,SchoolLevel,IsLibraryAvailable
0,(I.E.C) INDIAN EVANGALISTIC CRUSADE MISSION PR...,PS,Not Available
1,10TH MILE GOVERNMENT PRIMARY SCHOOL,PS,Not Available
2,AAPGAZI GOVERNMENT PRIMARY SCHOOL,PS,Not Available
3,AARUBOTEY GOVERNMENT PRIMARY SCHOOL,PS,Not Available
4,AHO-YANGTAM GOVERNMENT PRIMARY SCHOOL,PS,Not Available


In [ ]:
# Close the database connection
conn.close()